In [1]:
import chromadb
from chromadb.api.models.Collection import Collection
from llama_cpp import Llama
import qdrant_client
from qdrant_client import QdrantClient
from typing import List, Optional, Union
from langchain.vectorstores import Qdrant, Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)

In [2]:
collection = "all-documents"
model = "models/saiga2_7b_gguf/model-q2_K.gguf"
EMBEDDER_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
llama_model = Llama(
    model_path=model,
    n_ctx=2000,
    n_parts=1,
)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDER_NAME)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/saiga2_7b_gguf/model-q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q2_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q3_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q3_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q3_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q3_K     [ 14336,  4096,     1,     1 

In [3]:
LOADER_MAPPING: dict = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

def load_single_document(file_path: str):
    """
    Загружаем один документ.
    :param file_path:
    :return:
    """
    ext: str = "." + file_path.rsplit(".", 1)[-1]
    assert ext in LOADER_MAPPING
    loader_class, loader_args = LOADER_MAPPING[ext]
    loader = loader_class(file_path, **loader_args)
    return loader.load()[0]

In [4]:
query = "Какие требования к актуализации ВНД"
path = "load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx"
documents: List[Document] = [load_single_document(path)]

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

ids: List[str] = [
    f"{doc.metadata['source'].split('/')[-1].replace('.txt', '')}{i}"
    for i, doc in enumerate(docs)
]

documents = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

Created a chunk of size 1117, which is longer than the specified 1000
Created a chunk of size 1975, which is longer than the specified 1000
Created a chunk of size 1494, which is longer than the specified 1000
Created a chunk of size 2952, which is longer than the specified 1000
Created a chunk of size 1492, which is longer than the specified 1000
Created a chunk of size 1099, which is longer than the specified 1000
Created a chunk of size 2596, which is longer than the specified 1000


In [ ]:
# LANGCHAIN QDRANT

In [5]:
client = qdrant_client.QdrantClient(path="./qdrant_langchain_test")
qdrant = Qdrant(
    client=client, 
    collection_name=collection, 
    embeddings=embeddings
)

In [7]:
qdrant = qdrant.from_documents(
    docs,
    embeddings,
    path="./qdrant_langchain_test",
    collection_name=collection,
)
qdrant

In [8]:
found_docs = qdrant.similarity_search(query)

print(found_docs[0].metadata)
print(found_docs[0].page_content)

{'source': 'load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx'}
использование стандартизованной терминологии (корпоративного глоссария);

процедуры и действия, предусмотренные документом, должны быть реально выполнимыми.

Для оптимизации сроков основного согласования ВНД целесообразно привлекать на этапе разработки документа работников подразделений – обязательных участников основного согласования, а также работников ООР для методологической поддержки.

Результатом этапа разработки ВНД является:

проект ВНД;

проект распорядительного документа о введении в действие ВНД;

пояснительная записка или краткая презентация о целесообразности разработки ВНД и его ключевых положениях (при необходимости).

По результатам разработки все проекты ВНД должны пройти процедуру предварительного согласования, включающую рассмотрение:

непосредственным руководителем Разработчика на соответствие содержания ВНД целям его разработки;


In [ ]:
# LANGCHAIN CHROMA

In [5]:
client = chromadb.PersistentClient(path="./chroma_langchain_test")
db = Chroma(
    client=client,
    collection_name=collection,
    embedding_function=embeddings,
)

In [6]:
db.get()

{'ids': ['24fdc8f4-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8f5-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8f6-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8f7-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8f8-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8f9-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8fa-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8fb-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8fc-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8fd-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8fe-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc8ff-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc900-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc901-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc902-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc903-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc904-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc905-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc906-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc907-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc908-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc909-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc90a-9b5d-11ee-92d7-a5ce4627ccaf',
  '24fdc90b-9b5d-11ee-92d7-

In [13]:
db = db.from_documents(
    docs, 
    embeddings,
    persist_directory="./chroma_langchain_test",
    collection_name=collection,
)

In [ ]:
db = db.

In [14]:
docs = db.similarity_search(query)

# print results
print(docs[0].metadata)
print(docs[0].page_content)

{'source': 'load_texts/ПН_РСК_001_02_Положение_о_системе_ВНД.docx'}
использование стандартизованной терминологии (корпоративного глоссария);

процедуры и действия, предусмотренные документом, должны быть реально выполнимыми.

Для оптимизации сроков основного согласования ВНД целесообразно привлекать на этапе разработки документа работников подразделений – обязательных участников основного согласования, а также работников ООР для методологической поддержки.

Результатом этапа разработки ВНД является:

проект ВНД;

проект распорядительного документа о введении в действие ВНД;

пояснительная записка или краткая презентация о целесообразности разработки ВНД и его ключевых положениях (при необходимости).

По результатам разработки все проекты ВНД должны пройти процедуру предварительного согласования, включающую рассмотрение:

непосредственным руководителем Разработчика на соответствие содержания ВНД целям его разработки;


In [12]:
# CHROMA

In [5]:
client: chromadb.PersistentClient = chromadb.PersistentClient(path="./chroma_test")
db: Collection = client.get_or_create_collection(collection)
db

Collection(name=all-documents)

In [6]:
db.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [7]:
docs = db.query(
    query_texts=[query],
    n_results=4
)

In [8]:
data: dict = {}
for doc, text in zip(docs["metadatas"][0], docs["documents"][0]):
    document: str = f'Документ - {doc["source"].split("/")[-1]}'
    if document in data:
        data[document] += "\n" + text
    else:
        data[document] = text
list_data: list = [f"{doc}\n\n{text}" for doc, text in data.items()]
print("\n\n\n".join(list_data))

Документ - ПН_РСК_001_02_Положение_о_системе_ВНД.docx

В Обществе предусмотрены следующие виды ВНД:

Таблица №1
Пример:
                                                                             Положение Отдела бюджетирования
Уровень 2                  Должностная инструкция                            Документ, закрепляющий трудовую функцию и перечень обязанностей, права и ответственность по конкретной должности, а также квалификационные требования, предъявляемые к занятию этой должности
Область применения: определение иерархии и видов внутренних нормативных документов, общих подходов и правил разработки, согласования, утверждения и введения в действие, актуализации, отмены и хранения ВНД.

Общие положения

Целью системы управления внутренними нормативными документами является снижение неопределенности и повышение стабильности процессов посредством регламентации деятельности.

Основными задачами системы внутренних нормативных документов являются: 

обеспечение уровня регламентации, 

In [ ]:
# QDRANT

In [7]:
client = QdrantClient(path="./qdrant_test", embedding_models=embeddings)

In [8]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name=collection,
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [9]:
client.get_collection(collection_name=collection)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [12]:
client.add(
    collection_name=collection,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

AssertionError: Collection have incompatible vector params: size=4 distance=<Distance.DOT: 'Dot'> hnsw_config=None quantization_config=None on_disk=None

In [10]:
search_result = client.query(
    collection_name=collection,
    query_text=query
)
print(search_result)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 77.7M/77.7M [01:35<00:00, 817kiB/s]


ValueError: Dense vector fast-bge-small-en is not found in the collection

In [11]:
client.search(
    collection_name=collection,
    query_vector=(query, [0.2, 0.1, 0.9, 0.7]),
    limit=3,
)

ValueError: Dense vector Какие требования к актуализации ВНД is not found in the collection